# CS 224N Final Project - Evaluating on CBT Dataset
By: Joseph O'Brien, Christopher Pondoc, Joseph Guman

In [1]:
import torch
print("Using GPU: " + str(torch.cuda.is_available()))

Using GPU: True


## Load in GPT-2 Model
Using HuggingFace Transformers

In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to("cuda")

## Load in WinoDict Dataset and Make dictionary of real words and corresponding fake words

In [3]:
import pandas as pd

# Import CSV and get the real to fake
rtf_df = pd.read_csv("datasets/realtofake.csv")
real_words_list = rtf_df["Real"].tolist()
fake_words_list = rtf_df["Fake"].tolist()

# Populate dictionary
real_to_fake_dict = {}
for i in range(len(real_words_list)):
    real_to_fake_dict[real_words_list[i]] = fake_words_list[i]

## Load in Children's Book Test Dataset
Also taken from HuggingFace

In [4]:
import pandas
cbt_df = pd.read_csv("updated_cbt_extract/updated_cbt_info.csv")

## Evaluating on One Example
Writing a function that is reusable and works for one example

In [9]:
def evaluate_cbt(example):
    # extract the sentence
    sentence = example["sentences"]
    sentence = sentence.replace('"','')
    sentence = sentence.replace("\n","")
    sentence = sentence[1:len(sentence)-1]

    # extract the question
    question = example["question"]

    # make the total query with the sentence and the question
    total_query = sentence + " " + question

    
    #replace words in example with winodict fake words (currently, we replace all possible identified examples)
    replace_words = example["winodict_words"]
    replace_words = replace_words[1:len(replace_words)-1].replace("'","").split(",")

    for word in replace_words:
        word = word.strip()
        word = " "+word+" "
        while word in total_query:
            total_query = total_query.replace(word," "+real_to_fake_dict[word.strip()]+" ")
    

    # get the answer options for the model into a list
    options = example["options"].replace("'","")
    options = options.replace("\n","")
    options = options[1:len(options)-1].split(" ")

    # get the answer to the problem
    answer = example["answer"]

    
    if answer in replace_words:
        answer = real_to_fake_dict[answer][0]
    
    

    # initialize best answer and best loss (will be lowest value)
    best_answer = ""
    best_loss = float("inf")

    # compute the loss for each option (using full scoring)
    for option in options:
        
        if option in replace_words:
            option = real_to_fake_dict[option][0]
        

        updated_query = total_query.replace("XXXXX", option)
        
        # Tokenize each string and produce labels
        updated_input = tokenizer(updated_query, return_tensors="pt").to("cuda")
        #print(updated_input)
        print(len(updated_input[0]))

        if len(updated_input[0]) > 1024:
            return -1

        current_loss = model(**updated_input, labels=updated_input["input_ids"].to("cuda")).loss
        
        if current_loss < best_loss:
            best_answer = option
            best_loss = current_loss
    return best_answer == answer 



## Evaluating CBT on GPT-2

In [10]:
# evaluating gpt-2 WITHOUT REPLACING words and saving indices and stats

correct_replaced_list = []
total_replaced_list = []

correct = 0 
total = 0 
for index, row in cbt_df.iterrows():
    print(index)

    update = evaluate_cbt(row)
    if update == -1:
        continue
    if update == 1:
        correct_replaced_list.append(index)
    total_replaced_list.append(index)
    correct += update
    total += 1
    break

correct_replaced_list = str(correct_replaced_list)
total_replaced_list = str(total_replaced_list)

"""
with open("cbt_eval_info/NONreplaced_standardgpt2_correct", "w") as file1:
    file1.write(correct_replaced_list)

with open("cbt_eval_info/NONreplaced_standardgpt2_total", "w") as file2:
    file2.write(total_replaced_list)
"""

print("")
print("Number correct: ", correct)
print("Total: ", total)
print("Correct percentage: ",correct/total)

information_list = str([correct, total, correct/total])

"""
with open("cbt_eval_info/NONreplaced_standardgpt2_statistics", "w") as file3:
    file3.write(information_list)
"""



0
Encoding(num_tokens=760, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=760, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=761, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=759, attributes=[i

'\nwith open("cbt_eval_info/NONreplaced_standardgpt2_statistics", "w") as file3:\n    file3.write(information_list)\n'

In [6]:
# evaluating gpt-2 AFTER REPLACING words and saving indices and stats

"""
with open("cbt_eval_info/NONreplaced_standardgpt2_correct", "r") as f:
    correct_indices = f.read()

correct_indices = correct_indices[1:len(correct_indices)-1].split(", ")
for i in range(len(correct_indices)):
    correct_indices[i] = int(correct_indices[i])
"""

correct_replaced_list = []
total_replaced_list = []

correct = 0 
total = 0 
for index, row in cbt_df.iterrows():
    print(index)

    update = evaluate_cbt(row)
    if update == -1:
        continue
    if update == 1:
        correct_replaced_list.append(index)
    total_replaced_list.append(index)
    correct += update
    total += 1

return 
correct_replaced_list = str(correct_replaced_list)
total_replaced_list = str(total_replaced_list)

"""
with open("cbt_eval_info/REPLACED_standardgpt2_correct", "w") as file1:
    file1.write(correct_replaced_list)

with open("cbt_eval_info/REPLACED_standardgpt2_total", "w") as file2:
    file2.write(total_replaced_list)
"""

print("")
print("Number correct: ", correct)
print("Total: ", total)
print("Correct percentage: ",correct/total)

information_list = str([correct, total, correct/total])

"""
with open("cbt_eval_info/REPLACED_standardgpt2_statistics", "w") as file3:
    file3.write(information_list)
"""

0
1
2
3


Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 1024). Running this sequence through the model will result in indexing errors


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


## Making the Adversarial Dataset

In [4]:
with open("cbt_eval_info/REPLACED_standardgpt2_correct", "r") as f:
    replaced_correct_indices = f.read()
replaced_correct_indices = replaced_correct_indices[1:len(replaced_correct_indices)-1].split(", ")
for i in range(len(replaced_correct_indices)):
    replaced_correct_indices[i] = int(replaced_correct_indices[i])

with open("cbt_eval_info/NONreplaced_standardgpt2_correct", "r") as f:
    nonreplaced_correct_indices = f.read()
nonreplaced_correct_indices = nonreplaced_correct_indices[1:len(nonreplaced_correct_indices)-1].split(", ")
for i in range(len(nonreplaced_correct_indices)):
    nonreplaced_correct_indices[i] = int(nonreplaced_correct_indices[i])


with open("cbt_eval_info/REPLACED_standardgpt2_total", "r") as f:
    replaced_total_indices = f.read()
replaced_total_indices = replaced_total_indices[1:len(replaced_total_indices)-1].split(", ")
for i in range(len(replaced_total_indices)):
    replaced_total_indices[i] = int(replaced_total_indices[i])

with open("cbt_eval_info/NONreplaced_standardgpt2_total", "r") as f:
    nonreplaced_total_indices = f.read()
nonreplaced_total_indices = nonreplaced_total_indices[1:len(nonreplaced_total_indices)-1].split(", ")
for i in range(len(nonreplaced_total_indices)):
    nonreplaced_total_indices[i] = int(nonreplaced_total_indices[i])



adversarial_list = []

for index in nonreplaced_correct_indices:
    #make sure that both analyzed this example
    if index not in replaced_total_indices:
        continue
    if index not in replaced_correct_indices:
        adversarial_list.append(index)

print(len(adversarial_list))

with open("cbt_eval_info/ADVERSARIAL_indices", "w") as f:
    f.write(str(adversarial_list))


930
